In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('MeteoOlsztyn').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/12 13:58:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_spark = spark.read.csv('open-meteo-53.44N20.25E193m.csv', header=True, inferSchema=True)
df_spark.show(df_spark.count(), False)

+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|time               |temperature_2m_c|rain_mm|snowfall_cm|snow_depth_m|weather_code|visibility_m|wind_speed_10m_km_h|wind_speed_80m_km_h|
+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|2023-12-12 00:00:00|1.8             |0.0    |0.0        |0.06        |45          |80.0        |4.2                |10.8               |
|2023-12-12 01:00:00|1.5             |0.0    |0.0        |0.06        |3           |100.0       |3.6                |9.2                |
|2023-12-12 02:00:00|1.1             |0.0    |0.0        |0.06        |3           |100.0       |2.9                |8.4                |
|2023-12-12 03:00:00|0.9             |0.0    |0.0        |0.06        |3           |100.0       |3.4                |8.6                |
|2023-12-12 04:00:00|0.9          

In [3]:
df_spark.describe().show()

23/12/12 13:59:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-------------------+--------------------+--------------------+------------------+------------------+-------------------+-------------------+
|summary|  temperature_2m_c|            rain_mm|         snowfall_cm|        snow_depth_m|      weather_code|      visibility_m|wind_speed_10m_km_h|wind_speed_80m_km_h|
+-------+------------------+-------------------+--------------------+--------------------+------------------+------------------+-------------------+-------------------+
|  count|               336|                336|                 336|                 336|               336|               336|                336|                336|
|   mean|2.7657738095238114|0.11083333333333331|0.019702380952380954|0.022499999999999954|           27.1875|13595.892857142857|  21.38214285714286|   36.7169642857143|
| stddev| 2.731013916363612| 0.2477659383349636|0.059984332235318995| 0.02750847901848535|28.762599445559015|10371.654415754001| 10.348363899402473| 14.880

### Filtry

In [4]:
# temperatura powyżej 1 stopnia C
df_spark.filter('temperature_2m_c>=1').show()

+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|               time|temperature_2m_c|rain_mm|snowfall_cm|snow_depth_m|weather_code|visibility_m|wind_speed_10m_km_h|wind_speed_80m_km_h|
+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|2023-12-12 00:00:00|             1.8|    0.0|        0.0|        0.06|          45|        80.0|                4.2|               10.8|
|2023-12-12 01:00:00|             1.5|    0.0|        0.0|        0.06|           3|       100.0|                3.6|                9.2|
|2023-12-12 02:00:00|             1.1|    0.0|        0.0|        0.06|           3|       100.0|                2.9|                8.4|
|2023-12-12 10:00:00|             1.0|    0.0|        0.0|        0.06|           3|        80.0|                7.4|               15.8|
|2023-12-12 11:00:00|             

In [6]:
df_spark.filter('temperature_2m_c>=1').select(['time', 'rain_mm']).show()

+-------------------+-------+
|               time|rain_mm|
+-------------------+-------+
|2023-12-12 00:00:00|    0.0|
|2023-12-12 01:00:00|    0.0|
|2023-12-12 02:00:00|    0.0|
|2023-12-12 10:00:00|    0.0|
|2023-12-12 11:00:00|    0.0|
|2023-12-12 12:00:00|    0.0|
|2023-12-12 13:00:00|    0.0|
|2023-12-12 14:00:00|    0.1|
|2023-12-12 15:00:00|    0.0|
|2023-12-12 16:00:00|    0.0|
|2023-12-12 17:00:00|    0.0|
|2023-12-12 18:00:00|    0.0|
|2023-12-12 19:00:00|    0.0|
|2023-12-12 20:00:00|    0.1|
|2023-12-12 21:00:00|    0.3|
|2023-12-12 22:00:00|    0.2|
|2023-12-12 23:00:00|    0.0|
|2023-12-13 00:00:00|    0.0|
|2023-12-13 01:00:00|    0.0|
|2023-12-13 13:00:00|    0.0|
+-------------------+-------+


In [5]:
df_spark.filter((df_spark['temperature_2m_c'] > 1) & (df_spark['rain_mm'] > 0)).show()

+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|               time|temperature_2m_c|rain_mm|snowfall_cm|snow_depth_m|weather_code|visibility_m|wind_speed_10m_km_h|wind_speed_80m_km_h|
+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|2023-12-12 14:00:00|             1.2|    0.1|        0.0|        0.05|          51|       120.0|                7.9|               15.8|
|2023-12-12 20:00:00|             1.3|    0.1|        0.0|        0.05|          51|        60.0|                5.9|               14.1|
|2023-12-12 21:00:00|             1.2|    0.3|        0.0|        0.05|          61|        80.0|                5.8|               13.2|
|2023-12-12 22:00:00|             1.1|    0.2|        0.0|        0.05|          53|        80.0|                5.2|               12.1|
|2023-12-16 00:00:00|             

In [ ]:
# znaki specjalne: and - &, or - |, not ~

In [7]:
df_spark.filter(~(df_spark['temperature_2m_c'] > 1) & (df_spark['rain_mm'] > 0)).show()

+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|               time|temperature_2m_c|rain_mm|snowfall_cm|snow_depth_m|weather_code|visibility_m|wind_speed_10m_km_h|wind_speed_80m_km_h|
+-------------------+----------------+-------+-----------+------------+------------+------------+-------------------+-------------------+
|2023-12-13 17:00:00|             0.4|    0.1|       0.14|        0.05|          71|      2820.0|               10.4|               21.2|
|2023-12-13 21:00:00|             0.2|    0.1|       0.14|        0.05|          71|        20.0|               11.9|               23.2|
|2023-12-13 22:00:00|             0.2|    0.1|       0.14|        0.05|          77|        20.0|               11.9|               23.0|
|2023-12-15 23:00:00|             0.9|    0.4|        0.0|        0.07|          53|       460.0|               14.6|               27.6|
|2023-12-19 17:00:00|             

### Groupby and Aggregate Functions

In [8]:
df_spark.groupBy('rain_mm').sum().show()

+-------+---------------------+-------------------+-------------------+-------------------+-----------------+-----------------+------------------------+------------------------+
|rain_mm|sum(temperature_2m_c)|       sum(rain_mm)|   sum(snowfall_cm)|  sum(snow_depth_m)|sum(weather_code)|sum(visibility_m)|sum(wind_speed_10m_km_h)|sum(wind_speed_80m_km_h)|
+-------+---------------------+-------------------+-------------------+-------------------+-----------------+-----------------+------------------------+------------------------+
|    0.0|    513.3000000000001|                0.0|  3.499999999999999|  6.489999999999988|             3036|        3226560.0|                  4225.1|       7416.899999999997|
|    0.2|   49.599999999999994| 1.9999999999999998|                0.0|               0.17|              578|         145420.0|                   177.9|      335.99999999999994|
|   0.05|                 10.5|0.15000000000000002|                0.0|                0.0|              153| 

In [9]:
df_spark.groupBy('rain_mm').mean().show()

+-------+---------------------+-------------------+--------------------+--------------------+------------------+------------------+------------------------+------------------------+
|rain_mm|avg(temperature_2m_c)|       avg(rain_mm)|    avg(snowfall_cm)|   avg(snow_depth_m)| avg(weather_code)| avg(visibility_m)|avg(wind_speed_10m_km_h)|avg(wind_speed_80m_km_h)|
+-------+---------------------+-------------------+--------------------+--------------------+------------------+------------------+------------------------+------------------------+
|    0.0|    2.261233480176212|                0.0|0.015418502202643168|   0.028590308370044|13.374449339207048|14213.920704845816|      18.612775330396477|      32.673568281938316|
|    0.2|    4.959999999999999|0.19999999999999998|                 0.0|               0.017|              57.8|           14542.0|                   17.79|      33.599999999999994|
|   0.05|                  3.5|0.05000000000000001|                 0.0|                 0

In [10]:
df_spark.groupBy('rain_mm').count().show()

+-------+-----+
|rain_mm|count|
+-------+-----+
|    0.0|  227|
|    0.2|   10|
|   0.05|    3|
|    0.7|    3|
|    0.1|   17|
|    0.6|    9|
|    0.8|    6|
|   0.19|    3|
|   0.35|    3|
|    1.3|    3|
|    0.4|    9|
|   0.15|    9|
|   0.17|    6|
|    1.5|    3|
|   0.02|    6|
|    0.3|   16|
|   0.06|    3|
+-------+-----+


In [12]:
df_spark.agg({'rain_mm': 'sum', 'snowfall_cm': 'sum'}).show()

+----------------+------------------+
|sum(snowfall_cm)|      sum(rain_mm)|
+----------------+------------------+
|            6.62|37.239999999999995|
+----------------+------------------+


In [13]:
spark.stop()